In [ ]:
pip install git+https://github.com/r-trimbour/atacnet --force-reinstall --no-deps

In [ ]:
pip install snapatac2

In [ ]:
pip install ipywidgets

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=r".*Reordering categories will always return a new Categorical object.*")
warnings.filterwarnings("ignore", category=FutureWarning, message=r".*is_categorical_dtype is deprecated and will be removed in a future version.*")

In [ ]:
import atacnet as an
import scanpy as sc

In [ ]:
import numpy as np
import pandas as pd
import anndata as ad

import scipy as sp
from scipy import linalg
from sklearn.datasets import make_sparse_spd_matrix
import matplotlib.pyplot as plt

In [ ]:
sc.datasets.pbmc3k()

# Test with sparse covariance true matrix as GT

### 0. Create fake AnnData

In [ ]:
# Create fake single-cell atac-seq data
nb_cells = 300
nb_chr = 10
nb_regions_per_chr = 200
between_reg = 2000
size_reg = 50

counts = []
for chr in range(nb_chr):
    counts.append(pd.DataFrame(np.random.randint(0,100, size=(nb_cells, nb_regions_per_chr)),
                        index=['Cell_'+j for j in map(str, range(nb_cells))],
                        columns=['chr'+str(chr)+'_'+str(i)+'_'+str(i+size_reg) for i in range(1, nb_regions_per_chr*between_reg+1, between_reg)]))
atac = ad.AnnData(pd.concat(counts, axis=1))

In [ ]:
distance_threshold = 50000

### 1. Add region position in AnnData.obs

In [ ]:
an.add_region_infos(atac)

### 2. Replace random data with fake cov matrix

In [ ]:
n_samples, n_features = 300, nb_regions_per_chr

prng = np.random.RandomState(0)
prec = make_sparse_spd_matrix(
    n_features, alpha=0.99, smallest_coef=0.4, largest_coef=0.7, random_state=prng
)
cov = linalg.inv(prec)

#cov with only potential connections
possible_co = sp.sparse.csr_matrix(an.atacnet.get_distances_regions(atac)<distance_threshold/2)[:cov.shape[0],:cov.shape[1]]
possible_co = sp.sparse.coo_matrix(possible_co).toarray() + sp.sparse.coo_matrix(possible_co).toarray().T 
cov = np.eye(len(cov))*np.diag(cov) + possible_co*cov 
d = np.sqrt(np.diag(cov))
cov /= d
cov /= d[:, np.newaxis]
prec *= d
prec *= d[:, np.newaxis]
X = prng.multivariate_normal(np.zeros(n_features), cov, size=n_samples)
X -= X.mean(axis=0)
X /= X.std(axis=0)

X_ = np.concatenate([X]*nb_chr, axis=1)
atac.X = np.abs(X_)
atac.X = np.where(X_<0, 0, X_)

### 2.A. Remove Null rows

In [ ]:
sc.pp.filter_genes(atac, min_cells=1)
atac

### 2.B. Compute pseudocells

In [ ]:
# To come soon !

### 3 Calculate co-accessibility

In [ ]:
a = sp.sparse.coo_matrix([[1,-2],[0,0]])
b = sp.sparse.coo_matrix([[1, 1,],[-1, 0]])

In [ ]:
sp.sparse.csr_matrix.multiply((sp.sparse.csr_matrix(a<0)).astype(int), (sp.sparse.csr_matrix(b)>0).astype(int)).todense()

In [ ]:
(b-a).data

In [ ]:
a_ = set([i for i in zip(a.row, a.col)])
b_ = set([i for i in zip(b.row, b.col)])

In [ ]:
sp.sparse.csr_matrix.divide(a,b)

In [ ]:
a[(1,1),(1,1)]

In [ ]:
if (False - True):
    print('ha')

In [ ]:
an.compute_atac_network(
    atac, #metacells,
    window_size=distance_threshold,
    unit_distance = 1000,
    distance_constraint=distance_threshold/2,
    n_samples=50,
    n_samples_maxtry=100,
    max_alpha_iteration=60
)

*Can be stored externally using sliding_graphica_lasso*

In [ ]:
atac.X = sp.sparse.csr_matrix(atac.X)
# atac.X = atac.X.toarray()

In [ ]:
final_score = an.sliding_graphical_lasso(
    atac,
    window_size=distance_threshold,
    unit_distance = 1000,
    distance_constraint=distance_threshold/2,
    n_samples=50,
    n_samples_maxtry=100,
    max_alpha_iteration=500
)
atac.varp['atac_network'] = final_score

### 3. B. Extract list of edges

In [ ]:
an.extract_atac_links(atac) #metacells)

### 4. Plot comparison between co-accessibility scores and covariance matrix used to generate the data

In [ ]:
def diag_block_mat_slicing(L):
    shp = L[0].shape
    N = len(L)
    r = range(N)
    out = np.zeros((N,shp[0],N,shp[1]),dtype=int)
    out[r,:,r,:] = L
    return out.reshape(np.asarray(shp)*N)

corrected = final_score.toarray()[:200, :200]
corrected = np.where(corrected <= 0, corrected, corrected)
corrected = corrected - np.diag(corrected)*np.eye(len(cov))

cov_ = cov - np.diag(cov)*np.eye(len(cov))

fig, ax = plt.subplots(1, 2, figsize=(10,10))
ax[0].imshow(np.abs(corrected))
ax[1].imshow(np.where(cov_<=0, cov_, cov_))


print((corrected[:20,:20]>0).sum()/(400), (cov_[:20,:20]>0).sum()/(400),)

In [ ]:
corrected = final_score.toarray()[:200, :200]
corrected = np.where(corrected <= 0, 0, 1)


fig, ax = plt.subplots(1, 2, figsize=(10,10))
ax[0].imshow(corrected)
ax[1].imshow(np.where(cov<=0, 0, 1))

print((corrected[10,10]>0).sum()/(100), (cov[:10,:10]>0).sum()/(100))

In [ ]:
df = pd.DataFrame([cov.flatten(), corrected.flatten()]).transpose()

In [ ]:
final_score